In [1]:
import pandas as pd
import os
os.getcwd()

'/Users/leesunhong/2019capstone/all_resume'

In [2]:
def read_txt_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
        # remove_quotes
        data = data.replace("‘", " ")
        data = data.replace("’", " ")
        data = data.replace("“", " ")
        data = data.replace("”", " ")
        data = data.replace("`", " ")
        data = data.replace("\'", " ")
        data = data.replace("\"", " ")
        
        
    return data

In [3]:
challenge_data = read_txt_data('dataset/challenge_large.txt')
creative_data = read_txt_data('dataset/creative_large.txt')
teamwork_data = read_txt_data('dataset/teamwork_large.txt')


In [15]:
import csv


with open('dataset/challenge.csv', 'w') as out_file:
    docs_challenge = csv.writer(out_file)
    docs_challenge.writerow(["document"])
    lines = challenge_data.split(".")
    for i in lines:
        docs_challenge.writerow([i])

with open('dataset/creative.csv', 'w') as out_file:
    docs_creative = csv.writer(out_file)
    docs_creative.writerow(["document"])
    lines = creative_data.split(".")
    for i in lines:
        docs_creative.writerow([i])

with open('dataset/teamwork.csv', 'w') as out_file:
    docs_teamwork = csv.writer(out_file)
    docs_teamwork.writerow(["document"])
    lines = teamwork_data.split(".")
    for i in lines:
        docs_teamwork.writerow([i])
        


In [16]:
ch = pd.read_csv("dataset/challenge.csv", encoding="utf-8")
cr = pd.read_csv("dataset/creative.csv", encoding="utf-8")
t = pd.read_csv("dataset/teamwork.csv", encoding="utf-8")

In [20]:
ch["label"]= 1
ch.head()

,document,label
0,도전 이에 김태호 PD가 준비 중인 새로운 프로그램이 MBC 무한도전 의 빈...,1
1,무한도전 종영 이후 1년,1
2,"박성현, LPGA투어 2개 대회 연속 우승 도전 세계 1위를 탈환한 박성현은 3월...",1
3,헬스와 빅데이터 등 미래 먹거리 산업에 대한 혁신적 아이디어와 도전정신이 부족했다,1
4,빅 석세스 에 도전하는 벤처생태계 조성이 절실하다는 것이다,1


In [21]:
cr["label"]= 2
cr.head()

,document,label
0,창의\t올해 처음으로 열린 계절발표회에서는 황금연 경희대 교수가 대행 선사의 수행관...,2
1,김인유 기자 = 경기 수원시가 2022년 개장하려는 수원수목원 을 독창적인 수목...,2
2,페스티벌 공연 모습 KT&G는 상상마당 이라는 문화 플랫폼을 만들어 독창적이면...,2
3,이처럼 세계적 작가의 원작 소설을 독창적으로 재해석하여 전 세계 평단과 관객의 사...,2
4,경기 수원시가 오는 2022년 개장하려는 수원수목원 을 독창적인 수목원으로 조성하...,2


In [22]:
t["label"]= 3
t.head()

,document,label
0,"팀워크\t사진=삼화네트웍스 제공 열혈사제 가 배우, 스태프 할 것 없이 끈끈한 팀...",3
1,열혈사제 배우들이 완벽한 팀워크를 보여주고 있다,3
2,완벽한 하모니를 이루는 열혈사제 배우들의 호흡은 끈끈한 팀워크에서 나온다고 할...,3
3,"유사시 특정 지역에서의 전시 임무 수행 절차를 숙달하고, 연합작전 수행을 위한 팀워...",3
4,해치 의 정일우-권율-박훈-한상진이 최강 팀워크의 정석을 뽐내는 모습이 포착됐다,3


In [23]:
data = ch.append(cr)

In [24]:
data = data.append(t)

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split

In [26]:
train, test = train_test_split(data, test_size=0.33, random_state=42)

In [27]:
train["label"] = pd.Categorical(train["label"])

/Users/leesunhong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
train.groupby("label").count()

,document
label,
1,29042
2,26219
3,29360


In [29]:
test.groupby("label").count()

,document
label,
1,14391
2,12882
3,14407


In [30]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

f = open('dataset/stopwords.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
stopwords = list()

for row in reader:
    stopwords.append(row[0])

In [31]:
okt = Okt()

def get_noun(raw, pos=["Noun"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 정제 과정
            stem=True    # stemming 정제 과정
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]


In [32]:
cv = CountVectorizer(tokenizer=get_noun)

In [33]:
tdm = cv.fit_transform(train["document"])

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf_svm = Pipeline([('vect', CountVectorizer(tokenizer=get_noun)),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(train["document"], train["label"])

/Users/leesunhong/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [35]:
predicted_svm = text_clf_svm.predict(test["document"])
np.mean(predicted_svm == test["label"])

0.8177303262955854

In [36]:
 text_clf_svm.predict(["팀워크를 중요시하는 사람입니다."])

array([3])

In [37]:
text_clf_svm.predict(["도전적인 목표를 가지고 살았습니다"])

array([1])

In [38]:
text_clf_svm.predict(["창의적인 활동을 좋아합니다"])

array([2])

In [41]:
text_clf_svm.predict(["[열정을 갖고 끊임없이 노력하는 사람] 우선 주어진 일에 실패를 두려워하지 않고 끊임없이 도전하는 것입니다. 노력은 배신하지 않는다고 생각합니다. 주어진 일에 관한 분야의 역량을 함양하고자 시간 투자를 계속해서 쏟으며, 그 일을 반복하여 제 것으로 만듭니다. 저는 지금도 현재 진행형이며, 미래에도 멈추지 않고 달려 나가는 인재가 되고 싶습니다. 항상 배우고자 하고, 저 자신에게 나태해지지 않는 마음가짐으로 모든 일을 대할 것입니다. 팀 내부에서 저의 역할이 무엇인지 먼저 이해하고 팀원 간의 협력을 가장 중요시 여길 것입니다. 팀 프로젝트를 진행 할 때 가장 중요한 것은 팀원 간의 조화라고 생각하기 때문입니다. 이를 통해 어제보다 오늘의 제가, 오늘보다 내일의 제가 더 나은 사람이길 바랍니다. 안주하는 삶에 만족하지 않고 저의 가능성을 최대로 발휘하고 싶습니다. CJ올리브네트웍스에서도 발전하는 IT서비스와 시스템을 개발, 운영하기 위해 항상 새로운 도전에 주저없이 뛰어들겠습니다."])


array([1])